# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [64]:
cities_df = pd.read_csv("Resources/city_results.csv", dtype="object", encoding="utf-8")
cities_df.head()


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Pentecoste,BR,-3.7928,-39.2703,79.68,57,10,6.31,1627865354
1,Saldanha,ZA,-33.0117,17.9442,48.25,87,40,8.16,1627865354
2,Regen,DE,48.9667,13.1333,57.2,96,100,3.02,1627865355
3,Paamiut,GL,61.994,-49.6678,45.66,93,100,8.59,1627865355
4,Ushuaia,AR,-54.8,-68.3,37.17,87,34,8.41,1627865104


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#setup of heatmap plotting criteria 
location_df = cities_df[["Lat", "Lng"]].astype(float)
humidity_df = cities_df['Humidity'].astype(float)

In [66]:
# use the plotting criteria and make a heatmap, and also try and make it look professional-ish
fig = gmaps.figure(zoom_level=2,center=(20,15))

heat_layer = gmaps.heatmap_layer(location_df, weights=humidity_df, 
                                 dissipating=False,point_radius=7, opacity =.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
#tried to clean up the data and was getting errors on the type, running a for-loop to update the column types
#not sure if i'll need to do this for the API calls, but will have it handy in case
columns_to_change = ["Max Temp","Humidity","Cloudiness"]

for column in columns_to_change:
    cities_df[f"{column}"] = cities_df[f"{column}"].astype('float64')
cities_df.dtypes

City           object
Country        object
Lat            object
Lng            object
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed     object
Date           object
dtype: object

In [68]:
max_filtered_df = cities_df[(cities_df['Max Temp'] >= 72) & (cities_df['Max Temp'] <= 82)]
cloudy_filtered_df = max_filtered_df[max_filtered_df["Cloudiness"]<=75]
cloudy_filtered_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Pentecoste,BR,-3.7928,-39.2703,79.68,57.0,10.0,6.31,1627865354
11,Ponta do Sol,PT,32.6667,-17.1,72.23,89.0,2.0,4.03,1627865357
14,Ribeira Grande,PT,38.5167,-28.7,72.82,95.0,16.0,13.67,1627865358
36,São Félix do Xingu,BR,-6.6447,-51.995,78.48,46.0,57.0,2.46,1627865364
37,George Town,MY,5.4112,100.3354,78.89,91.0,20.0,1.99,1627865066
...,...,...,...,...,...,...,...,...,...
659,Humberto de Campos,BR,-2.5983,-43.4611,74.57,85.0,32.0,3.33,1627865540
664,Pokhara,NP,28.2333,83.9833,74.34,89.0,68.0,0.94,1627865542
674,Shīeli,KZ,44.1667,66.75,79.41,16.0,0.0,10.54,1627865545
679,Pitimbu,BR,-7.4706,-34.8086,73.99,83.0,26.0,16.58,1627865546


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [77]:
hotel_df=pd.DataFrame(cloudy_filtered_df).reset_index()
hotel_df['Hotel Name']=" "
hotel_df

,index,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,0,Pentecoste,BR,-3.7928,-39.2703,79.68,57.0,10.0,6.31,1627865354,
1,11,Ponta do Sol,PT,32.6667,-17.1,72.23,89.0,2.0,4.03,1627865357,
2,14,Ribeira Grande,PT,38.5167,-28.7,72.82,95.0,16.0,13.67,1627865358,
3,36,São Félix do Xingu,BR,-6.6447,-51.995,78.48,46.0,57.0,2.46,1627865364,
4,37,George Town,MY,5.4112,100.3354,78.89,91.0,20.0,1.99,1627865066,
...,...,...,...,...,...,...,...,...,...,...,...
94,659,Humberto de Campos,BR,-2.5983,-43.4611,74.57,85.0,32.0,3.33,1627865540,
95,664,Pokhara,NP,28.2333,83.9833,74.34,89.0,68.0,0.94,1627865542,
96,674,Shīeli,KZ,44.1667,66.75,79.41,16.0,0.0,10.54,1627865545,
97,679,Pitimbu,BR,-7.4706,-34.8086,73.99,83.0,26.0,16.58,1627865546,


In [78]:
# geocoordinates
target_lat = hotel_df["Lat"]
target_lng = hotel_df["Lng"]
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary

for index,row in hotel_df.iterrows():
    
    params = {
        "location": f"{target_lat},{target_lng}",
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotels = response.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f'Found Hotel {hotels["results"][0]["name"]}')
    except (KeyError, IndexError):
        print(f"Could Not Find Hotel for attempted {target_lat}, {target_lng}")

Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.

Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.

Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.

Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.

Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.2333
96    44.1667
97    -7.4706
98     9.7359
Name: Lat, Length: 99, dtype: object, 0     -39.2703
1        -17.1
2        -28.7
3      -51.995
4     100.3354
        ...   
94    -43.4611
95     83.9833
96       66.75
97    -34.8086
98    -75.5263
Name: Lng, Length: 99, dtype: object
Could Not Find Hotel for attempted 0     -3.7928
1     32.6667
2     38.5167
3     -6.6447
4      5.4112
       ...   
94    -2.5983
95    28.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
